In [1]:
import sys, pathlib
ROOT = pathlib.Path().resolve().parent
sys.path.append(str(ROOT))

import json
import os

from normalizer.data_utils import normalizer as data_utils_normalizer
normalizer = data_utils_normalizer

# Cell 1
from datasets import load_dataset, Dataset, DatasetDict, Value
from pathlib import Path
import csv, re


/home/ubuntu/jtm/whisper-tune/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from datasets import load_dataset, DatasetDict, Value
from pathlib import Path
import csv, re

SPECIAL_WORDS_PATH = "/home/ubuntu/jtm/open_asr_leaderboard/transformers/global_dict_500.csv"

with open(SPECIAL_WORDS_PATH, "r", encoding="utf-8") as f:
    special_words = [line.strip().rstrip(",") for line in f if line.strip()]
    normalized_special_words = {normalizer(word) for word in special_words}

In [4]:
def has_special_word(x):
    text = normalizer(x["text"])
    words_in_text = set(text.split())
    matched_words = words_in_text.intersection(normalized_special_words)
    return len(matched_words) > 0

In [5]:
ds_stream = load_dataset(
    "aquavoice/cleaned_dataset_full_2x_en_resplit",
    split="test",
    streaming=False,          # <- important
)


In [6]:
filtered_ds = ds_stream.filter(has_special_word)
filtered_ds

Filter: 100%|██████████| 36894/36894 [01:43<00:00, 356.79 examples/s]


Dataset({
    features: ['audio', 'text', 'session_id', 'speaker_id', 'duration', 'whisper_text', 'elevenlabs_text', 'speechmatics_text', 'confidence', 'language'],
    num_rows: 9198
})

In [ ]:
filtered_ds = ds_stream.filter(has_special_word)
filtered_ds

Filter: 100%|██████████| 36894/36894 [01:43<00:00, 356.79 examples/s]


Dataset({
    features: ['audio', 'text', 'session_id', 'speaker_id', 'duration', 'whisper_text', 'elevenlabs_text', 'speechmatics_text', 'confidence', 'language'],
    num_rows: 9198
})

Dataset({
    features: ['audio', 'text', 'session_id', 'speaker_id', 'duration', 'whisper_text', 'elevenlabs_text', 'speechmatics_text', 'confidence', 'language'],
    num_rows: 9198
})

In [8]:
new_ds_name = "jmci/aispeak-v1"
to_push = DatasetDict({"test": filtered_ds})
to_push

DatasetDict({
    test: Dataset({
        features: ['audio', 'text', 'session_id', 'speaker_id', 'duration', 'whisper_text', 'elevenlabs_text', 'speechmatics_text', 'confidence', 'language'],
        num_rows: 9198
    })
})

In [9]:
to_push.push_to_hub(new_ds_name, private=True, max_shard_size="500MB")

Uploading the dataset shards: 100%|██████████| 12/12 [04:22<00:00, 21.90s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/jmci/aispeak-v1/commit/b6aa0d0232bf699059bba795b0b3c42e9c4ec59e', commit_message='Upload dataset', commit_description='', oid='b6aa0d0232bf699059bba795b0b3c42e9c4ec59e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/jmci/aispeak-v1', endpoint='https://huggingface.co', repo_type='dataset', repo_id='jmci/aispeak-v1'), pr_revision=None, pr_num=None)